# Data Preparation

In [1]:
!python --version

Python 3.11.4


In [2]:
import yaml
import pickle
import warnings
import numpy as np
import pandas as pd
import synapseclient
from tqdm import tqdm

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

case_study_root = "../data/case_study"

Please go to synapse and log in to get the authentication token. Without `config.yaml` you will not be able to execute the following code.

In [3]:
def get_auth_token_from_yaml(file_path='../static/config.yaml'):
    with open(file_path, 'r') as file:
        config_data = yaml.safe_load(file)
        auth_token = config_data.get('auth_token')
        return auth_token

auth_token = get_auth_token_from_yaml()
syn = synapseclient.Synapse()
syn.login(authToken=auth_token)

Welcome, fuad021!



### Temporal Gene Expression Profile Selection

In [4]:
# raw temporal gene extression dataset
syn57367276 = syn.get(entity='syn57367276')

raw_df = pd.read_csv(syn57367276.path, sep='\t')
raw_df = raw_df.fillna(0)
raw_df = raw_df.set_index('Gene_symbol')

raw_df

,GLSS.SM.R109.TP.01R.RNA.I4HTPP,GLSS.19.0274.TP.01R.RNA.ZZD8PZ,GLSS.CU.R005.TP.01R.RNA.LOURPF,GLSS.SM.R060.TP.01R.RNA.ZI9YVM,GLSS.MD.0017.R2.01R.RNA.L5HMEO,GLSS.HF.47E9.R1.01R.RNA.7W5GMH,TCGA.TQ.A8XE.TP.11R.RNA.0YPLAZ,GLSS.LU.00C2.TP.01R.RNA.2QRUOC,GLSS.HF.CB66.TP.01R.RNA.9MRJMX,GLSS.LU.00B9.R1.01R.RNA.1IVJNJ,TCGA.06.0210.TP.01R.RNA.JMBENW,TCGA.DU.7304.R1.12R.RNA.OMXRQS,GLSS.SN.0001.R1.01R.RNA.PZAYRK,GLSS.MD.0022.TP.01R.RNA.EDOJNO,GLSS.19.0267.R1.01R.RNA.NCCNA9,TCGA.TQ.A7RK.R2.11R.RNA.BJMJ93,GLSS.HF.DF35.TP.01R.RNA.IH1LVD,TCGA.06.0221.R1.11R.RNA.UCUX3T,GLSS.MD.0048.R1.01R.RNA.RVQU3C,GLSS.HF.6504.TP.01R.RNA.5OSIEV,GLSS.CU.P024.TP.01R.RNA.8JTL48,GLSS.HF.4F0A.TP.01R.RNA.GG6762,GLSS.SM.R107.TP.01R.RNA.ZSYP37,GLSS.CU.R009.R1.01R.RNA.QREUYE,TCGA.FG.5965.R2.11R.RNA.UCBSIY,TCGA.FG.5963.R1.12R.RNA.1EJ1SV,GLSS.CU.P104.R1.01R.RNA.L5UZ2L,GLSS.HF.B30B.R2.01R.RNA.K9X72E,GLSS.MD.0046.R1.01R.RNA.Y7WTHG,GLSS.SN.0017.TP.01R.RNA.FX5HYW,GLSS.MD.0017.R1.01R.RNA.8FU0BP,GLSS.SM.R070.TP.01R.RNA.QWG60O,GLSS.LU.00C7.R1.01R.RNA.898J17,GLSS.SM.R101.R1.01R.RNA.7ATA59,GLSS.MD.0012.R1.01R.RNA.GB101L,GLSS.HF.2869.TP.01R.RNA.2H6MOF,GLSS.MD.0027.R1.01R.RNA.E771C5,GLSS.MD.0044.TP.01R.RNA.2ARML9,GLSS.CU.P069.TP.01R.RNA.CVOV9T,GLSS.SM.R101.TP.02R.RNA.V4TRVD,TCGA.19.0957.R1.01R.RNA.7WHBFU,GLSS.SN.0009.TP.01R.RNA.1DY2G0,GLSS.19.0279.R2.01R.RNA.G733J7,GLSS.HK.0004.R1.01R.RNA.1PAW5U,GLSS.LU.00C7.TP.01R.RNA.7QCLSN,GLSS.19.0271.R1.01R.RNA.ZGQFM3,GLSS.19.0280.R1.01R.RNA.CZ9ZTK,TCGA.DH.A669.TP.12R.RNA.XG7CTS,GLSS.MD.0041.R1.01R.RNA.YF565D,GLSS.CU.R012.TP.01R.RNA.21UZH1,GLSS.CU.P053.TP.01R.RNA.DAMM5M,GLSS.CU.P101.TP.01R.RNA.2II6XG,GLSS.LX.0304.R1.01R.RNA.68YN4U,GLSS.SF.0038.TP.01R.RNA.TSEK1M,GLSS.SM.R083.TP.01R.RNA.RZOKGO,GLSS.SM.R064.TP.01R.RNA.W6PIW2,GLSS.HF.2829.R1.01R.RNA.D71HI8,GLSS.SN.0002.TP.01R.RNA.F7DBXG,GLSS.LU.00C4.R1.01R.RNA.O1YTL6,GLSS.SM.R080.TP.01R.RNA.2N8175,GLSS.MD.0027.R2.01R.RNA.1RO1LP,GLSS.SM.R091.TP.01R.RNA.RUF3X9,GLSS.SM.R106.TP.01R.RNA.P5JI35,GLSS.SN.0010.TP.01R.RNA.E97Z4T,GLSS.CU.R002.TP.01R.RNA.7CO09H,GLSS.MD.LP02.R1.01R.RNA.6ZQS8U,GLSS.SM.R091.R1.01R.RNA.M356AA,TCGA.DU.5870.R1.12R.RNA.GIZUWN,GLSS.SN.0013.TP.01R.RNA.5952RG,GLSS.CU.P102.R1.01R.RNA.QODUFA,GLSS.CU.P013.R3.01R.RNA.V38ZQ5,TCGA.19.4065.TP.01R.RNA.UGJCJV,GLSS.SM.R082.R1.01R.RNA.9RBAL7,GLSS.19.0272.R1.01R.RNA.MW1TDZ,GLSS.MD.0012.TP.01R.RNA.49NNFK,GLSS.MD.0035.R1.01R.RNA.ZXNEZ3,GLSS.SN.0008.R1.01R.RNA.AK537W,GLSS.MD.0049.TP.01R.RNA.HFW0T1,GLSS.MD.0019.TP.01R.RNA.MQF7QS,GLSS.HF.2829.TP.01R.RNA.90I02Y,GLSS.SN.0017.R1.01R.RNA.OEK8D6,TCGA.TQ.A7RK.R1.11R.RNA.PKFXB3,GLSS.HF.9A7A.TP.01R.RNA.KGGESL,GLSS.HF.2919.R1.01R.RNA.R6L8SO,GLSS.LU.0B10.TP.01R.RNA.LNPBXB,TCGA.DU.6397.TP.11R.RNA.NYNT65,GLSS.SM.R093.R1.01R.RNA.56NKHA,TCGA.06.0190.TP.01R.RNA.S2074S,GLSS.HF.F922.R1.01R.RNA.XNFKVB,TCGA.06.0125.TP.01R.RNA.RCQ5QS,GLSS.HF.753F.R4.01R.RNA.B36DGR,GLSS.HF.57AE.R2.01R.RNA.NAVDEK,GLSS.MD.0020.TP.01R.RNA.NVJSCE,GLSS.MD.0009.TP.01R.RNA.TX2UR7,GLSS.HK.0001.TP.01R.RNA.BYEBS0,GLSS.HF.DF35.R1.01R.RNA.KMLJLA,GLSS.SM.R087.TP.01R.RNA.M99T61,GLSS.MD.0026.TP.01R.RNA.7PF2HB,GLSS.HF.3118.R1.01R.RNA.HNBVM0,GLSS.SM.R066.TP.01R.RNA.G9A6CK,TCGA.06.0211.R1.02R.RNA.3YQ1G4,GLSS.SN.0006.TP.01R.RNA.N5ULKZ,GLSS.LX.0083.R1.01R.RNA.VS46JF,GLSS.HF.6504.R1.01R.RNA.BSOQQB,TCGA.06.0152.R1.01R.RNA.1IIR6O,TCGA.DU.6407.R2.11R.RNA.F6PNTL,GLSS.CU.P003.R1.01R.RNA.2OJ4BA,GLSS.SM.R067.TP.01R.RNA.JYXPKW,GLSS.HF.4D9A.TP.01R.RNA.C8NU0J,GLSS.SM.R081.TP.01R.RNA.4GQW1O,GLSS.CU.R001.R1.01R.RNA.SRFKW8,GLSS.LU.00B9.TP.01R.RNA.S9XA8D,GLSS.SM.R061.TP.01R.RNA.AZSPBW,GLSS.MD.0014.R1.01R.RNA.WIUIL4,GLSS.LU.0B13.R1.01R.RNA.CZ1AHH,GLSS.SM.R101.TP.01R.RNA.G7G4Q5,GLSS.HF.9A7A.R1.01R.RNA.RQWI9I,GLSS.SM.R095.TP.01R.RNA.5T2HT4,GLSS.SN.0013.R2.01R.RNA.A9XVF7,GLSS.CU.P021.TP.01R.RNA.Q4LSD8,GLSS.CU.P069.R1.01R.RNA.95RLAP,GLSS.HF.2998.R1.01R.RNA.2HE04E,GLSS.HF.2934.TP.01R.RNA.WUQKHB,GLSS.SM.R107.R1.01R.RNA.ID07M4,GLSS.SM.R107.R1.02R.RNA.1K9XUJ,GLSS.LX.0357.R2.01R.RNA.HBTTUU,GLSS.HF.EE77.R2.01R.RNA.ZNRTM7,GLSS.HK.0003.R1.01R.RNA.34VZDE,GLSS.LX.0083.R2.01R.RNA.EXTF60,

In [5]:
def get_case_df(raw_df, patient_ids, case_name):
    case_ids = sorted([id for id in list(raw_df.columns) if id.split('.')[2] in patient_ids])
    case_df = raw_df[case_ids]
    print(case_ids, case_df.shape)
    case_df.columns = [case_name+'_'+column_name.split('.')[3] for column_name in case_df.columns]
    case_df = case_df.groupby(case_df.columns, axis=1).median()
    return case_df

baseline_ids = ['5965', 'F922', 'A7RK', 'R064']
baseline_df = get_case_df(raw_df, baseline_ids, 'baseline')
aggressive_df = get_case_df(raw_df, ["0279"], 'aggressive')
nonaggressive_df = get_case_df(raw_df, ["0027"], 'nonaggressive')

print(baseline_df.shape, aggressive_df.shape, nonaggressive_df.shape)
baseline_df

['GLSS.HF.F922.R1.01R.RNA.XNFKVB', 'GLSS.HF.F922.R2.01R.RNA.5SZ198', 'GLSS.HF.F922.TP.01R.RNA.43U22A', 'GLSS.SM.R064.R1.01R.RNA.GLLBN3', 'GLSS.SM.R064.R2.01R.RNA.TUJCN7', 'GLSS.SM.R064.TP.01R.RNA.W6PIW2', 'TCGA.FG.5965.R1.11R.RNA.QSA7Y3', 'TCGA.FG.5965.R2.11R.RNA.UCBSIY', 'TCGA.FG.5965.TP.11R.RNA.0XBPNY', 'TCGA.TQ.A7RK.R1.11R.RNA.PKFXB3', 'TCGA.TQ.A7RK.R2.11R.RNA.BJMJ93', 'TCGA.TQ.A7RK.TP.11R.RNA.LRGF1D'] (35438, 12)
['GLSS.19.0279.R1.01R.RNA.J1DZC9', 'GLSS.19.0279.R2.01R.RNA.G733J7', 'GLSS.19.0279.TP.01R.RNA.R37O1O'] (35438, 3)
['GLSS.MD.0027.R1.01R.RNA.E771C5', 'GLSS.MD.0027.R2.01R.RNA.1RO1LP', 'GLSS.MD.0027.TP.01R.RNA.9J243C'] (35438, 3)
(35438, 3) (35438, 3) (35438, 3)


,baseline_R1,baseline_R2,baseline_TP
Gene_symbol,,,
CMSS1,31.207177,35.164799,24.341754
STX16,35.996421,44.218382,20.716331
TLE4,20.813337,22.299578,10.835135
LRRN1,31.667442,16.577943,53.999371
YKT6,92.464768,82.431033,75.375286
...,...,...,...
BMP2,5.453945,6.221065,27.504200
EP300,12.412165,13.973550,12.963900
ATP6V0E1P3,0.000000,0.000000,0.000000


In [6]:
# filtering expressed genes with minimum threshold
all_case_df = pd.concat([baseline_df, aggressive_df, nonaggressive_df], axis=1)
all_case_df = all_case_df.loc[(all_case_df >= 0.001).all(axis=1)]

baseline_df = all_case_df[all_case_df.columns[all_case_df.columns.str.contains("baseline")]]
aggressive_df = all_case_df[all_case_df.columns[all_case_df.columns.str.contains("aggressive") & ~all_case_df.columns.str.contains("non")]]
nonaggressive_df = all_case_df[all_case_df.columns[all_case_df.columns.str.contains("nonaggressive")]]

all_case_df

,baseline_R1,baseline_R2,baseline_TP,aggressive_R1,aggressive_R2,aggressive_TP,nonaggressive_R1,nonaggressive_R2,nonaggressive_TP
Gene_symbol,,,,,,,,,
CMSS1,31.207177,35.164799,24.341754,26.446038,27.000944,32.025694,31.395261,48.859002,44.117700
STX16,35.996421,44.218382,20.716331,99.860372,104.422161,126.156318,20.050733,21.480300,45.597264
TLE4,20.813337,22.299578,10.835135,23.209358,19.561900,31.615145,72.895632,96.452925,65.074070
LRRN1,31.667442,16.577943,53.999371,2.167500,0.433401,5.336610,121.089577,101.425790,137.481300
YKT6,92.464768,82.431033,75.375286,52.902599,65.192139,59.786585,11.058250,13.540128,9.911100
...,...,...,...,...,...,...,...,...,...
UQCRFS1,82.462700,62.024200,65.626250,54.498700,58.399200,50.797400,68.277000,105.298000,60.563400
C4orf32,3.717895,4.994350,2.917710,0.268489,0.327187,0.151905,4.348520,4.666130,4.126970
MTRNR2L9,36.739885,82.480730,47.055200,0.430776,0.616574,0.785486,111.565000,63.872200,98.041800


### Differentially Expressed Gene Selection

In [7]:
def calculate_FC(df):
    name = df.columns[0].split('_')[0]
    temp_df = df.copy()
    temp_df.loc[:, f"{name}_R1/TP"] = temp_df[f"{name}_R1"] / temp_df[f"{name}_TP"]
    temp_df.loc[:, f"{name}_R2/TP"] = temp_df[f"{name}_R2"] / temp_df[f"{name}_TP"]
    temp_df.drop(columns=[f"{name}_R1", f"{name}_R2", f"{name}_TP"], inplace=True)
    return temp_df

baseline_df = calculate_FC(baseline_df)
aggressive_df = calculate_FC(aggressive_df)
nonaggressive_df = calculate_FC(nonaggressive_df)

print(baseline_df.shape, aggressive_df.shape, nonaggressive_df.shape)
baseline_df

(11090, 2) (11090, 2) (11090, 2)


,baseline_R1/TP,baseline_R2/TP
Gene_symbol,,
CMSS1,1.282043,1.444629
STX16,1.737587,2.134470
TLE4,1.920912,2.058080
LRRN1,0.586441,0.307003
YKT6,1.226725,1.093608
...,...,...
UQCRFS1,1.256551,0.945113
C4orf32,1.274251,1.711736
MTRNR2L9,0.780783,1.752850


In [8]:
def get_DEGs(df, c, up_th, down_th):
    glist_up = list(df[df[c] >= up_th].index)
    glist_down = list(df[df[c] <= dn_th].index)
    glist = set(glist_up + glist_down)
    print(f"# selected genes: up:{len(glist_up)}, down:{len(glist_down)}, total:{len(glist)}")
    return glist

# selecting differentially expressed genes if FC >= 1.5 or FC =< 0.5
up_th = 1.5
dn_th = abs(1-(up_th-1))
print(f"up-regulation th: {up_th}\ndonw-regulation th: {dn_th}\n")

gene_sets = []
for df in [baseline_df, aggressive_df, nonaggressive_df]:
    name = df.columns[0].split('_')[0]
    for c in [f"{name}_R1/TP", f"{name}_R2/TP"]:
        gene_sets.append(get_DEGs(df, c, up_th, dn_th))

with open(f'{case_study_root}/differentially_expressed_genes/DEGs.pkl', 'wb') as f:
    pickle.dump(gene_sets, f)

up-regulation th: 1.5
donw-regulation th: 0.5

# selected genes: up:2409, down:362, total:2771
# selected genes: up:3808, down:315, total:4123
# selected genes: up:745, down:923, total:1668
# selected genes: up:985, down:1101, total:2086
# selected genes: up:1727, down:3208, total:4935
# selected genes: up:2842, down:2583, total:5425


In [9]:
with open(f'{case_study_root}/differentially_expressed_genes/DEGs.pkl', 'rb') as f:
    gene_sets = pickle.load(f)

set_baseline_R1TP = set(gene_sets[0])
set_baseline_R2TP = set(gene_sets[1])
set_aggressive_R1TP = set(gene_sets[2])
set_aggressive_R2TP = set(gene_sets[3])
set_nonaggressive_R1TP = set(gene_sets[4])
set_nonaggressive_R2TP = set(gene_sets[5])

len(set_baseline_R1TP), len(set_baseline_R2TP), len(set_aggressive_R1TP), len(set_aggressive_R2TP), len(set_nonaggressive_R1TP), len(set_nonaggressive_R2TP)

(2771, 4123, 1668, 2086, 4935, 5425)

### Gene-set, Network, and Pathway Analysis (GNPA)
Retrieving pathways for DEGs using PAGER API

In [10]:
!wget -qq https://discovery.informatics.uab.edu/PAGER/application/download/PAGER.py -O PAGER.py

In [11]:
import PAGER as pager
PAGER = pager.PAGER()

def get_PAGs(gene_list):
    PAG_result = PAGER.run_pager(
        gene_list,
        source = ["WikiPathway_2021"],
        Type='P',
        minSize=1,
        maxSize=2000,
        similarity = 0.05,
        overlap = 1,
        nCoCo = 0,
        pValue = 0.05,
        FDR = 0.5
    )
    return PAG_result

pag_bR1 = get_PAGs(set_baseline_R1TP)
pag_bR2 = get_PAGs(set_baseline_R2TP)
pag_aR1 = get_PAGs(set_aggressive_R1TP)
pag_aR2 = get_PAGs(set_aggressive_R2TP)
pag_nR1 = get_PAGs(set_nonaggressive_R1TP)
pag_nR2 = get_PAGs(set_nonaggressive_R2TP)

print(pag_bR1.shape, pag_bR2.shape, pag_aR1.shape, pag_aR2.shape, pag_nR1.shape, pag_nR2.shape)
pag_bR1.head()

(302, 15) (533, 15) (117, 15) (140, 15) (460, 15) (560, 15)


,GS_ID,NAME,SOURCE,GS_SIZE,ORGANISM,DESCRIPTION,LINK,TYPE,MULTI_N,OLAP,COCO_V2,SIMILARITY_SCORE,pvalue,Rank,pFDR
0,WAG002718,Neutrophil degranulation,WikiPathway_2021,474,Homo sapiens,Neutrophil degranulation,https://www.wikipathways.org/index.php/Pathway...,P,1333,133,172.7292014,.0903083748223834781382534465740349050362,1.802010e-50,1,7.081899e-48
1,WAG002628,VEGFA-VEGFR2 signaling pathway,WikiPathway_2021,438,Homo sapiens,VEGFA-VEGFR2 signaling pathway,https://www.wikipathways.org/index.php/Pathway...,P,1333,116,980.8100106,.0829712696366962075211812798674869014877,1.978119e-41,2,7.754225e-39
2,WAG003212,Malignant pleural mesothelioma,WikiPathway_2021,430,Homo sapiens,Malignant pleural mesothelioma,https://www.wikipathways.org/index.php/Pathway...,P,1333,97,1643.647427,.0701162790697674418604651162790697674419,7.439607e-29,3,2.908886e-26
3,WAG003032,PodNet: protein-protein interactions in the po...,WikiPathway_2021,301,Homo sapiens,PodNet: protein-protein interactions in the po...,https://www.wikipathways.org/index.php/Pathway...,P,1333,78,843.8242674,.0726689258916504104554624198563239435682,1.242073e-27,4,4.844085e-25
4,WAG002393,Focal adhesion,WikiPathway_2021,199,Homo sapiens,Focal adhesion,https://www.wikipathways.org/index.php/Pathway...,P,1333,60,4929.347674,.0767963528985131844790965134953116095944,2.503145e-25,5,9.737235e-23


In [12]:
sub_directory = "geneset_network_and_pathway_analysis"

pag_bR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_baseline_R1_TP.csv", index=False)
pag_bR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_baseline_R2_TP.csv", index=False)
pag_aR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_aggressive_R1_TP.csv", index=False)
pag_aR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_aggressive_R2_TP.csv", index=False)
pag_nR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_nonaggressive_R1_TP.csv", index=False)
pag_nR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_nonaggressive_R2_TP.csv", index=False)

### Pathway Networks
Retrieving m-type PAG-2-PAG relationships

In [13]:
def get_m_relationships(pag_df):
    """retrieve the m-type relationships of PAGs using a list of PAG IDs"""
    return PAGER.pathInt(list(pag_df["GS_ID"]))

m_bR1 = get_m_relationships(pag_bR1)
m_bR2 = get_m_relationships(pag_bR2)
m_aR1 = get_m_relationships(pag_aR1)
m_aR2 = get_m_relationships(pag_aR2)
m_nR1 = get_m_relationships(pag_nR1)
m_nR2 = get_m_relationships(pag_nR2)

print(m_bR1.shape, m_bR2.shape, m_aR1.shape, m_aR2.shape, m_nR1.shape, m_nR2.shape)
m_bR1.head()

(1488, 7) (6220, 7) (208, 7) (260, 7) (5520, 7) (7602, 7)


,GS_A_ID,GS_A_SIZE,GS_B_ID,GS_B_SIZE,OLAP,SIMILARITY,PVALUE
0,WAG003212,430,WAG003224,153,55,.15522875816993464,149.90937335538615
1,WAG002393,199,WAG002267,81,37,.2131687242798354,141.99332515949845
2,WAG002393,199,WAG002440,21,12,.16043956043956045,49.9520463399183
3,WAG002414,90,WAG002096,63,18,.16380952380952382,73.94134202903723
4,WAG002414,90,WAG002440,21,13,.22993197278911565,66.35607680082285


In [14]:
for m in [m_bR1, m_bR2, m_aR1, m_aR2, m_nR1, m_nR2]:
    print(f'Unique PAGs: {len(set(m["GS_A_ID"]))}')

sub_directory = "pathway_networks"

m_bR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_baseline_R1_TP.csv", index=False)
m_bR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_baseline_R2_TP.csv", index=False)
m_aR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_aggressive_R1_TP.csv", index=False)
m_aR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_aggressive_R2_TP.csv", index=False)
m_nR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_nonaggressive_R1_TP.csv", index=False)
m_nR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_nonaggressive_R2_TP.csv", index=False)

Unique PAGs: 239
Unique PAGs: 463
Unique PAGs: 66
Unique PAGs: 91
Unique PAGs: 408
Unique PAGs: 492


### Differential Pathway analysis

Here, we'll be calculating weighted fold change for each pathways with the equation below:

$PAG_{WFC} = \frac{\sum_{i=1}^n W_i * FC_i}{\sum_{i=1}^n W_i}$

where, $W_i = RPScore(Gene_i)$, $FC_i = FoldChange(Gene_i)$ and $n = NumberOfGenesInPAG$.

These wFC along with adjusted p-values (pFDR) describe the states of the pathways comprehensively. Thus, different colors will be associated with different states of the pathways.

In [15]:
%%time

def weightedFC(wfc, w):
    if len(wfc):
        return round(sum(wfc)/sum(w), 4)
    else:
        return float(0)

def get_pagFC(pag_df, mem_df, fc_df, fc_col):
    pag_IDs = []
    pag_wfc = []
    pag_pvalue = []
    pag_pFDR = []

    # retrieve the genes (members) of the pathways using a list of IDs
    PAG_member = PAGER.pathMember(list(set(pag_df["GS_ID"])))

    for pagID in tqdm(PAG_member["GS_ID"].unique()):
        # retrieve RP-ranked genes with RP-score of the given PAG_IDs
        PAGRankedGenes = PAGER.pagRankedGene(pagID)
        PAGRankedGenes["RP_SCORE"] = PAGRankedGenes["RP_SCORE"].astype(float)

        ws = []
        wfc = []
        for gene in PAGRankedGenes["GENE_SYM"]:
            if gene in fc_df.index:
                w = PAGRankedGenes[PAGRankedGenes["GENE_SYM"]==gene]["RP_SCORE"].values[0]
                fc = fc_df[fc_df.index==gene][fc_col].values[0]
                if not np.isnan(w):
                    ws.append(w)
                    wfc.append(w*fc)

        pag_IDs.append(pagID)
        pag_wfc.append(weightedFC(wfc, ws))
        pag_pFDR.append(pag_df[pag_df["GS_ID"]==pagID]["pFDR"].values[0])

    pagFC_df = pd.DataFrame({
        "GS_ID": pag_IDs,
        "wFC": pag_wfc,
        "pFDR": pag_pFDR
    })

    return pagFC_df

pag_bR1 = get_pagFC(pag_bR1, m_bR1, baseline_df, "baseline_R1/TP")
pag_bR2 = get_pagFC(pag_bR2, m_bR2, baseline_df, "baseline_R2/TP")
pag_aR1 = get_pagFC(pag_aR1, m_aR1, aggressive_df, "aggressive_R1/TP")
pag_aR2 = get_pagFC(pag_aR2, m_aR2, aggressive_df, "aggressive_R2/TP")
pag_nR1 = get_pagFC(pag_nR1, m_nR1, nonaggressive_df, "nonaggressive_R1/TP")
pag_nR2 = get_pagFC(pag_nR2, m_nR2, nonaggressive_df, "nonaggressive_R2/TP")

100%|██████████| 560/560 [05:10<00:00,  1.81it/s]

CPU times: user 1min 58s, sys: 1.09 s, total: 1min 59s
Wall time: 19min 14s


In [16]:
pag_bR1.head()

,GS_ID,wFC,pFDR
0,WAG001976,1.3186,0.006331
1,WAG001977,1.6080,0.075830
2,WAG002000,2.0673,0.000030
3,WAG002011,1.0627,0.240331
4,WAG002018,1.7749,0.084468


In [17]:
sub_directory = "differential_pathway_analysis"

pag_bR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_baseline_R1_TP.csv", index=False)
pag_bR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_baseline_R2_TP.csv", index=False)
pag_aR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_aggressive_R1_TP.csv", index=False)
pag_aR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_aggressive_R2_TP.csv", index=False)
pag_nR1.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_nonaggressive_R1_TP.csv", index=False)
pag_nR2.to_csv(f"{case_study_root}/{sub_directory}/wikipathway_nonaggressive_R2_TP.csv", index=False)